# Test Notebook

This notebook provides a method of testing the `Project` class methods. It allows you to

1. Create a sample manifest and, if necessary, insert it into the database.
2. Run some find and delete operations on the database records.
3. Create an instance of your project using the sample manifest.
4. Run some tests on the project using the `Project` class methods.

In order to run this notebook, you should first start MongoDB. Then, in the `config/config.py` file, configure the url for the MongoDB client and the paths to your local template directory (where the template folders are stored) and your local workspace directory (where project folders are stored).

## Import Configuration and Project Class

In [ ]:
%run project/Project.py

## Sample Data Name

Enter the name for a sample manifest for which you wish to create a Project object.

In [ ]:
name = 'test'

## Add Sample Data

In [ ]:
# Change these values before running the cell to execute the database queries
insert_new_record = False
update_existing_record = False


# Place a sample manifest in the database - Configure the name below.
manifest = {
    'name': 'test-save-new'
}

# Insert a new manifest
if insert_new_record == True:
    projects_db.insert_one(manifest)
    print('The new manifest was added to the database.')
    
# From now on, use _id to access the manifest
result = list(projects_db.find(manifest))
try:
    _id = result[0]['_id']
    manifest['_id'] = _id
except:
    pass

# Add Some Project Versions to the Manifest and Update
if update_existing_record == True:
    manifest['content'] = []
    now = datetime.today().strftime('%Y%m%d%H%M%S')
    manifest['content'].append(
        {
            'version_date': now,
            'version_number': 1,
            'version_name': now + '_v1_' + manifest['name']

        }
    )
    now = datetime.today().strftime('%Y%m%d%H%M%S')
    manifest['content'].append(
        {
            'version_date': now,
            'version_number': 2,
            'version_name': now + '_v2_' + manifest['name']

        }
    )
    projects_db.update_one({'_id': _id},
                           {'$set': {'content': manifest['content']}}, upsert=False)
    print('The sample manifest was updated.')

# Output when both database configs are False
if insert_new_record == False and update_existing_record == False:
    print('No action was taken.')

## Convenience Functions

In [ ]:
def delete_by_id(_id):
    projects_db.delete_one({'_id': _id})
    print(str(_id) + ' deleted.')

def delete_by_name(name):
    projects_db.delete_one({'name': name})
    print(name + ' deleted.')

def find_by_id(_id):
    result = list(projects_db.find({'_id': _id}))
    print(json.dumps(result, indent=2, sort_keys=False, default=JSON_UTIL))

def find_by_name(name):
    result = list(projects_db.find({'name': name}))
    print(json.dumps(result, indent=2, sort_keys=False, default=JSON_UTIL))

# Uncomment to run one of the functions in this cell
# delete_by_id(ObjectId('5cf9d819b370023f6079706a'))
# delete_by_name(name)
# find_by_id(ObjectId('5cf9d923b370023f6079706c'))


## Create a Project Object

**Important:** This does not create a project in the WMS or Workspace. It merely creates an instance of the `Project` class, which can then be manipulated by class methods.

In [ ]:
try:
    project = Project(manifest, config.TEMPLATES_DIR, config.WORKSPACE_DIR, config.TEMP_DIR)
    print('Project object created')
except:
    print('Could not create Project object.')

## Test Methods

This cell demonstrates `Project.print_manifest()`. You can test other `Project` class methods by inserting more cells below. All methods are documented at <a href="https://whatevery1says.github.io/projectclass/" target="_blank">https://whatevery1says.github.io/projectclass/</a>.

In [ ]:
# Display the Manifest
project.print_manifest()